In [1]:
# importando os arquivos tsv para processamento e exportação no formato parquet

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = SparkSession.builder.appName("MyApp").getOrCreate()

23/11/01 21:16:57 WARN Utils: Your hostname, DESKTOP-TCH95BD resolves to a loopback address: 127.0.1.1; using 172.20.29.145 instead (on interface eth0)
23/11/01 21:16:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/01 21:17:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# Importando arquivos Tsv e criando dfs, usando delimitador /t e header true

In [ ]:
basicos_filmes = spark.read.option("delimiter", "\t").option("header", "true").csv("file:/home/hadoop/imdb/title_basic.tsv")
diretor_escritor = spark.read.option("delimiter", "\t").option("header", "true").csv("file:/home/hadoop/imdb/title_crew.tsv")
filmes_principais = spark.read.option("delimiter", "\t").option("header", "true").csv("file:/home/hadoop/imdb/title_principal.tsv")
filmes_votos = spark.read.option("delimiter", "\t").option("header", "true").csv("file:/home/hadoop/imdb/title_rating.tsv")
nome_colaboradores = spark.read.option("delimiter", "\t").option("header", "true").csv("file:/home/hadoop/imdb/name.tsv")

In [ ]:
# Lendo arquivos e criando o df basicos_filmes

In [18]:
basicos_filmes = spark.read.option("delimiter", "\t").option("header", "true").csv("file:/home/hadoop/imdb/title_basic.tsv")
basicos_filmes.printSchema()
basicos_filmes.show(10)
contagem_de_dados = basicos_filmes.count()
contagem_de_colunas = len(basicos_filmes.columns)

# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,

O DataFrame tem 9 colunas.
O DataFrame tem 10265844 linhas (dados).


In [ ]:
# No campo tipo, contém filmes, curtas... vou trabalhar apenas com o valor MOVIE
# Agora, alterando o nome das colunas e filtrando o gênero que contém apenas DRAMA, pois não utilizarei Romance.
# Pelo fato de ter escolhido o Diretor Martin Scorsese

In [ ]:
# Alterando o nome das colunas

In [19]:
# Alterando o nome das colunas

basicos_filmes = basicos_filmes.withColumnRenamed("tconst", "id_filme") \
        .withColumnRenamed("titleType", "tipo") \
        .withColumnRenamed("primaryTitle", "titulo") \
        .withColumnRenamed("originalTitle", "titulo_original") \
        .withColumnRenamed("isAdult", "adulto") \
        .withColumnRenamed("startYear", "lancamento") \
        .withColumnRenamed("endYear", "fim_lancamento") \
        .withColumnRenamed("runtimeMinutes", "duracao_filme") \
        .withColumnRenamed("genres", "genero")

# Filtrando apenas o tipo com valor MOVIE e apenas o Gênero com valor DRAMA

basicos_filmes = basicos_filmes.filter((basicos_filmes["tipo"] == "movie") & (basicos_filmes["genero"].contains("Drama")))


basicos_filmes.printSchema()
basicos_filmes.show(10)
contagem_de_dados = basicos_filmes.count()
contagem_de_colunas = len(basicos_filmes.columns)

# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

root
 |-- id_filme: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- titulo: string (nullable = true)
 |-- titulo_original: string (nullable = true)
 |-- adulto: string (nullable = true)
 |-- lancamento: string (nullable = true)
 |-- fim_lancamento: string (nullable = true)
 |-- duracao_filme: string (nullable = true)
 |-- genero: string (nullable = true)

+---------+-----+--------------------+--------------------+------+----------+--------------+-------------+---------------+
| id_filme| tipo|              titulo|     titulo_original|adulto|lancamento|fim_lancamento|duracao_filme|         genero|
+---------+-----+--------------------+--------------------+------+----------+--------------+-------------+---------------+
|tt0000591|movie|    The Prodigal Son|   L'enfant prodigue|     0|      1907|            \N|           90|          Drama|
|tt0000615|movie|  Robbery Under Arms|  Robbery Under Arms|     0|      1907|            \N|           \N|          Drama|
|tt000063

O DataFrame tem 9 colunas.
O DataFrame tem 238636 linhas (dados).


In [ ]:
# salvando a tabela e exportando o df em parquet

In [21]:
# importando datetime para separar os diretórios por dia

from datetime import datetime
current_date = datetime.now().strftime("%Y/%m/%d")

# Definindo a variável do Output

output_path = f"file:/home/hadoop/imdb/parquet/basico_filmes_drama/{current_date}"

# Salvar o DataFrame como um arquivo Parquet

basicos_filmes.coalesce(1).write.saveAsTable(name="filmes_basico_drama",mode="overwrite",path=output_path,format="parquet")

print("Os dados foram salvos com sucesso como arquivo Parquet.")

Os dados foram salvos com sucesso como arquivo Parquet.


In [22]:
# Lendo o arquivo e criando o df diretor_escritor

In [23]:
diretor_escritor = spark.read.option("delimiter", "\t").option("header", "true").csv("file:/home/hadoop/imdb/title_crew.tsv")

diretor_escritor.printSchema()
diretor_escritor.show(10)
contagem_de_dados = diretor_escritor.count()
contagem_de_colunas = len(diretor_escritor.columns)

# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

root
 |-- tconst: string (nullable = true)
 |-- directors: string (nullable = true)
 |-- writers: string (nullable = true)

+---------+-------------------+---------+
|   tconst|          directors|  writers|
+---------+-------------------+---------+
|tt0000001|          nm0005690|       \N|
|tt0000002|          nm0721526|       \N|
|tt0000003|          nm0721526|       \N|
|tt0000004|          nm0721526|       \N|
|tt0000005|          nm0005690|       \N|
|tt0000006|          nm0005690|       \N|
|tt0000007|nm0005690,nm0374658|       \N|
|tt0000008|          nm0005690|       \N|
|tt0000009|          nm0085156|nm0085156|
|tt0000010|          nm0525910|       \N|
+---------+-------------------+---------+
only showing top 10 rows



O DataFrame tem 3 colunas.
O DataFrame tem 10265844 linhas (dados).


In [ ]:
# alterando o nome das colunas 

In [24]:
diretor_escritor = diretor_escritor.withColumnRenamed("tconst", "id_filme") \
        .withColumnRenamed("directors", "diretores") \
        .withColumnRenamed("writers", "escritores") \
        
diretor_escritor.printSchema()
diretor_escritor.show(10)
contagem_de_dados = diretor_escritor.count()
contagem_de_colunas = len(diretor_escritor.columns)

# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

root
 |-- id_filme: string (nullable = true)
 |-- diretores: string (nullable = true)
 |-- escritores: string (nullable = true)

+---------+-------------------+----------+
| id_filme|          diretores|escritores|
+---------+-------------------+----------+
|tt0000001|          nm0005690|        \N|
|tt0000002|          nm0721526|        \N|
|tt0000003|          nm0721526|        \N|
|tt0000004|          nm0721526|        \N|
|tt0000005|          nm0005690|        \N|
|tt0000006|          nm0005690|        \N|
|tt0000007|nm0005690,nm0374658|        \N|
|tt0000008|          nm0005690|        \N|
|tt0000009|          nm0085156| nm0085156|
|tt0000010|          nm0525910|        \N|
+---------+-------------------+----------+
only showing top 10 rows



O DataFrame tem 3 colunas.
O DataFrame tem 10265844 linhas (dados).


In [ ]:
# salvando a tabela e exportando o df em parquet

In [25]:
from datetime import datetime

current_date = datetime.now().strftime("%Y/%m/%d")

# Definindo a variável do Output

output_path = f"file:/home/hadoop/imdb/parquet/diretores_escritores/{current_date}"

# Salvar o DataFrame como um arquivo Parquet

diretor_escritor.coalesce(1).write.saveAsTable(name="diretor_escritor",mode="overwrite",path=output_path,format="parquet")

print("Os dados foram salvos com sucesso como arquivo Parquet.")

Os dados foram salvos com sucesso como arquivo Parquet.


In [ ]:
# Lendo o arquivo e criando o df filmes_principais

In [26]:
filmes_principais = spark.read.option("delimiter", "\t").option("header", "true").csv("file:/home/hadoop/imdb/title_principal.tsv")

filmes_principais.printSchema()
filmes_principais.show(10)
contagem_de_dados = filmes_principais.count()
contagem_de_colunas = len(filmes_principais.columns)

# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

root
 |-- tconst: string (nullable = true)
 |-- ordering: string (nullable = true)
 |-- nconst: string (nullable = true)
 |-- category: string (nullable = true)
 |-- job: string (nullable = true)
 |-- characters: string (nullable = true)

+---------+--------+---------+---------------+--------------------+----------+
|   tconst|ordering|   nconst|       category|                 job|characters|
+---------+--------+---------+---------------+--------------------+----------+
|tt0000001|       1|nm1588970|           self|                  \N|  ["Self"]|
|tt0000001|       2|nm0005690|       director|                  \N|        \N|
|tt0000001|       3|nm0374658|cinematographer|director of photo...|        \N|
|tt0000002|       1|nm0721526|       director|                  \N|        \N|
|tt0000002|       2|nm1335271|       composer|                  \N|        \N|
|tt0000003|       1|nm0721526|       director|                  \N|        \N|
|tt0000003|       2|nm1770680|       producer|    

O DataFrame tem 6 colunas.
O DataFrame tem 58797177 linhas (dados).


In [ ]:
# Alterando as colunas

In [30]:
from pyspark.sql.functions import col

filme_ator = filmes_principais.withColumnRenamed("tconst", "id_filme") \
        .withColumnRenamed("ordering", "ordem") \
        .withColumnRenamed("nconst", "id_nome") \
        .withColumnRenamed("category", "categoria") \
        .withColumnRenamed("job", "trabalho") \
        .withColumnRenamed("characters", "personagem") \

# selecionando a ordem de apresentação das colunas
filme_ator = filme_ator.select(col("id_filme"), col("id_nome"), col("ordem"), col("categoria"), col("personagem"), col("trabalho"))


filme_ator.printSchema()
filme_ator.show(10)
contagem_de_dados = filme_ator.count()
contagem_de_colunas = len(filme_ator.columns)

# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

root
 |-- id_filme: string (nullable = true)
 |-- id_nome: string (nullable = true)
 |-- ordem: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- personagem: string (nullable = true)
 |-- trabalho: string (nullable = true)

+---------+---------+-----+---------------+----------+--------------------+
| id_filme|  id_nome|ordem|      categoria|personagem|            trabalho|
+---------+---------+-----+---------------+----------+--------------------+
|tt0000001|nm1588970|    1|           self|  ["Self"]|                  \N|
|tt0000001|nm0005690|    2|       director|        \N|                  \N|
|tt0000001|nm0374658|    3|cinematographer|        \N|director of photo...|
|tt0000002|nm0721526|    1|       director|        \N|                  \N|
|tt0000002|nm1335271|    2|       composer|        \N|                  \N|
|tt0000003|nm0721526|    1|       director|        \N|                  \N|
|tt0000003|nm1770680|    2|       producer|        \N|            produ

O DataFrame tem 6 colunas.
O DataFrame tem 58797177 linhas (dados).


In [ ]:
# salvando a tabela e exportando o df em parquet

In [31]:
from datetime import datetime

current_date = datetime.now().strftime("%Y/%m/%d")

# Definindo a variável do Output

output_path = f"file:/home/hadoop/imdb/parquet/filme_ator/{current_date}"

# Salvar o DataFrame como um arquivo Parquet

filme_ator.coalesce(1).write.saveAsTable(name="filmes_atores",mode="overwrite",path=output_path,format="parquet")

print("Os dados foram salvos com sucesso como arquivo Parquet.")

Os dados foram salvos com sucesso como arquivo Parquet.


In [ ]:
# Lendo o arquivo e criando o df filmes_votos

In [35]:
filmes_votos = spark.read.option("delimiter", "\t").option("header", "true").csv("file:/home/hadoop/imdb/title_rating.tsv")

filmes_votos.printSchema()
filmes_votos.show(10)
contagem_de_dados = filmes_votos.count()
contagem_de_colunas = len(filmes_votos.columns)

# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

# contagem de linhas, 1e6 é a notação científica para um milhão
milhoes_de_linhas = contagem_de_dados / 1e6
print(f"O DataFrame tem {milhoes_de_linhas:.2f} milhões de linhas (dados).")

root
 |-- tconst: string (nullable = true)
 |-- averageRating: string (nullable = true)
 |-- numVotes: string (nullable = true)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    2003|
|tt0000002|          5.8|     269|
|tt0000003|          6.5|    1898|
|tt0000004|          5.5|     178|
|tt0000005|          6.2|    2682|
|tt0000006|          5.0|     183|
|tt0000007|          5.4|     839|
|tt0000008|          5.4|    2147|
|tt0000009|          5.3|     207|
|tt0000010|          6.9|    7331|
+---------+-------------+--------+
only showing top 10 rows



O DataFrame tem 3 colunas.
O DataFrame tem 1363146 linhas (dados).
O DataFrame tem 1.36 milhões de linhas (dados).


In [ ]:
# Alterando as colunas

In [36]:
from pyspark.sql.functions import col

filmes_votos = filmes_votos.withColumnRenamed("tconst", "id_filme") \
        .withColumnRenamed("averageRating", "avaliacao_media") \
        .withColumnRenamed("numVotes", "total_votos") \

filmes_votos.printSchema()
filmes_votos.show(10)
contagem_de_dados = filmes_votos.count()
contagem_de_colunas = len(filmes_votos.columns)

# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

root
 |-- id_filme: string (nullable = true)
 |-- avaliacao_media: string (nullable = true)
 |-- total_votos: string (nullable = true)

+---------+---------------+-----------+
| id_filme|avaliacao_media|total_votos|
+---------+---------------+-----------+
|tt0000001|            5.7|       2003|
|tt0000002|            5.8|        269|
|tt0000003|            6.5|       1898|
|tt0000004|            5.5|        178|
|tt0000005|            6.2|       2682|
|tt0000006|            5.0|        183|
|tt0000007|            5.4|        839|
|tt0000008|            5.4|       2147|
|tt0000009|            5.3|        207|
|tt0000010|            6.9|       7331|
+---------+---------------+-----------+
only showing top 10 rows



O DataFrame tem 3 colunas.
O DataFrame tem 1363146 linhas (dados).


In [ ]:
# salvando a tabela e exportando o df em parquet

In [37]:
from datetime import datetime

current_date = datetime.now().strftime("%Y/%m/%d")

# Definindo a variável do Output

output_path = f"file:/home/hadoop/imdb/parquet/filmes_votos/{current_date}"

# Salvar o DataFrame como um arquivo Parquet

filmes_votos.coalesce(1).write.saveAsTable(name="filmes_votos",mode="overwrite",path=output_path,format="parquet")

print("Os dados foram salvos com sucesso como arquivo Parquet.")

Os dados foram salvos com sucesso como arquivo Parquet.


In [ ]:
# Lendo o df nomes_colaboradores

In [38]:
nome_colaboradores = spark.read.option("delimiter", "\t").option("header", "true").csv("file:/home/hadoop/imdb/name.tsv")

nome_colaboradores.printSchema()
nome_colaboradores.show(10)
contagem_de_dados = nome_colaboradores.count()
contagem_de_colunas = len(nome_colaboradores.columns)

# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

# contagem de linhas, 1e6 é a notação científica para um milhão
milhoes_de_linhas = contagem_de_dados / 1e6
print(f"O DataFrame tem {milhoes_de_linhas:.2f} milhões de linhas (dados).")


root
 |-- nconst: string (nullable = true)
 |-- primaryName: string (nullable = true)
 |-- birthYear: string (nullable = true)
 |-- deathYear: string (nullable = true)
 |-- primaryProfession: string (nullable = true)
 |-- knownForTitles: string (nullable = true)

+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0072308,tt00319...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0038355,tt01170...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0054452,tt00564...|
|nm0000004|   John Belushi|     1949|     1982|actor,soundtrack,...|tt0072562,tt00804...|
|nm0000005| Ingmar Bergman|     1918|     2007|writer,director,a...|tt0050986,tt00839...|
|nm0000006| Ingr

O DataFrame tem 6 colunas.
O DataFrame tem 12954255 linhas (dados).
O DataFrame tem 12.95 milhões de linhas (dados).


In [ ]:
# Alterando o nome das colunas, a ordem e um novo valor para nulo

In [44]:
from pyspark.sql.functions import col, when

info_colaboradores = nome_colaboradores.withColumnRenamed("nconst", "id_nome") \
        .withColumnRenamed("primaryName", "nome") \
        .withColumnRenamed("birthYear", "ano_nascimento") \
        .withColumnRenamed("deathYear", "ano_falecimento") \
        .withColumnRenamed("primaryProfession", "profissao_principal") \
        .withColumnRenamed("knownForTitles", "participacao_filmes") \

info_colaboradores = info_colaboradores.select(col("id_nome"), col("nome"), col("participacao_filmes"), col("profissao_principal"), col("ano_nascimento"), col("ano_falecimento"))

# usar \\N para garantir que o caractere \ seja interpretado literalmente
info_colaboradores = info_colaboradores.withColumn("ano_falecimento", when(col("ano_falecimento") == "\\N", "vivo(a)").otherwise(col("ano_falecimento")))

info_colaboradores.printSchema()
info_colaboradores.show(10)
contagem_de_dados = info_colaboradores.count()
contagem_de_colunas = len(info_colaboradores.columns)

# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

root
 |-- id_nome: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- participacao_filmes: string (nullable = true)
 |-- profissao_principal: string (nullable = true)
 |-- ano_nascimento: string (nullable = true)
 |-- ano_falecimento: string (nullable = true)

+---------+---------------+--------------------+--------------------+--------------+---------------+
|  id_nome|           nome| participacao_filmes| profissao_principal|ano_nascimento|ano_falecimento|
+---------+---------------+--------------------+--------------------+--------------+---------------+
|nm0000001|   Fred Astaire|tt0072308,tt00319...|soundtrack,actor,...|          1899|           1987|
|nm0000002|  Lauren Bacall|tt0038355,tt01170...|  actress,soundtrack|          1924|           2014|
|nm0000003|Brigitte Bardot|tt0054452,tt00564...|actress,soundtrac...|          1934|        vivo(a)|
|nm0000004|   John Belushi|tt0072562,tt00804...|actor,soundtrack,...|          1949|           1982|
|nm0000005| Ingma

O DataFrame tem 6 colunas.
O DataFrame tem 12954255 linhas (dados).


In [ ]:
# salvando a tabela e exportando o df em parquet

In [45]:
from datetime import datetime

current_date = datetime.now().strftime("%Y/%m/%d")

# Definindo a variável do Output

output_path = f"file:/home/hadoop/imdb/parquet/info_colaboradores/{current_date}"

# Salvar o DataFrame como um arquivo Parquet

info_colaboradores.coalesce(1).write.saveAsTable(name="colaboradores_filmes",mode="overwrite",path=output_path,format="parquet")

print("Os dados foram salvos com sucesso como arquivo Parquet.")

Os dados foram salvos com sucesso como arquivo Parquet.


In [ ]:
# verificar depois esse codigo: 
from pyspark.sql.functions import col, when

# Substitua "\N" por "adaptado" na coluna "escritores"
diretor_escritor = diretor_escritor.withColumn("escritores", when(col("escritores") == "\\N", "adaptado").otherwise(col("escritores")))
